In [2]:
import requests
from pyquery import PyQuery as pq
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep

### Wrap 2020-2021 Data From China Banking and Insurance Regulatory Commission

#### Wrap Function

In [3]:
#Get page 1-3
def get_web(page):
    r = requests.get('http://www.cbirc.gov.cn/cn/static/data/DocInfo/SelectDocByItemIdAndChild/data_itemId=954,pageIndex={page},pageSize=18.json'.format(page=page))
    return r

In [18]:
#Get page 4-6
def get_web2(page):
    cookies = {
        'yfx_c_g_u_id_10006849': '_ck21121511293510955416158987436',
        'yfx_f_l_v_t_10006849': 'f_t_1639538975091__r_t_1639538975091__v_t_1639546463631__r_c_0',
    }

    headers = {
        'Proxy-Connection': 'keep-alive',
        'Accept': '*/*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'Referer': 'http://www.cbirc.gov.cn/cn/view/pages/ItemList.html?itemPId=953&itemId=954&itemUrl=ItemListRightList.html&itemName=%E7%BB%9F%E8%AE%A1%E4%BF%A1%E6%81%AF',
        'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8',
    }

    params = (
        ('itemId', '954'),
        ('pageSize', '18'),
        ('pageIndex', '{page}'.format(page=page)),
    )

    r = requests.get('http://www.cbirc.gov.cn/cbircweb/DocInfo/SelectDocByItemIdAndChild', headers=headers, params=params, cookies=cookies, verify=False)
    return r

In [5]:
def get_seperate_page(docId):
    url = 'http://www.cbirc.gov.cn/cn/view/pages/ItemDetail.html?docId={docId}&itemId=954&generaltype=0'.format(docId = docId)
    #doc = pq(url)
    return url

#### Get DocId Dict

In [ ]:
pattern = re.compile('.*保险业经营情况表')
def parse_page(json,dic):
    if json:
        items = json.get('data').get('rows')
        for item in items:
            if re.match(pattern,item.get('docSubtitle')):
                dic[item.get('docSubtitle')] = item.get('docId')
    #    print(dic)

dic = {}
for page in range(1,4):
    r = get_web(page)
    parse_page(r.json(),dic)
print(dic)

In [8]:
def get_file_last(path):
    dir_list = os.listdir(path)
    dir_list = sorted(dir_list,key=lambda x: os.path.getmtime(os.path.join(path, x)))
    return dir_list[-1]
get_file_last("/mnt/d/data_science/Insurance_report/data/")

'raw'

#### Download Data Function

In [3]:
#Driver Setting
raw_data_path = "/mnt/d/data_science/Insurance_report/data/raw/"
fp = webdriver.FirefoxProfile()
fp.set_preference("browser.download.folderList", 2)        
fp.set_preference("browser.download.dir", raw_data_path)
fp.set_preference("browser.download.manager.showWhenStarting",False)
fp.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel;application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")

def get_one_file(driver,key,val):
    count = len(os.listdir(raw_data_path))
    driver.get(get_seperate_page(val))
    driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[2]/div/div[5]/ul/li/a').click()
    if len(os.listdir(raw_data_path)) == count + 1:
        file = raw_data_path + get_file_last(raw_data_path)
        os.rename(file, raw_data_path + '{key}.xlsx'.format(key=key))
        print("Success")




In [ ]:
driver = webdriver.Firefox(firefox_profile=fp)
for key,val in dic.items():    
    try:
        print("The page is about {key}\n".format(key=key))
        get_one_file(driver,key,val)
        sleep(3)
    except:
        print(key,'\tFailed')
        get_one_file(driver,key,val)
        #continue

In [15]:
driver = webdriver.Firefox(firefox_profile=fp)
driver.get(get_seperate_page((dic['2021年5月保险业经营情况表'])))
driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[2]/div/div[5]/ul/li/a').click()

In [21]:
dic2 = {}
for page in range(4,7):
    r = get_web2(page)
    parse_page(r.json(),dic2)
dic2

{'2020年12月保险业经营情况表': 963080,
 '2020年11月保险业经营情况表': 952787,
 '2020年10月保险业经营情况表': 945039,
 '2020年9月保险业经营情况表': 937163,
 '2020年8月保险业经营情况表': 931640,
 '2020年7月保险业经营情况表': 925152,
 '2020年6月保险业经营情况表': 918275,
 '2020年5月保险业经营情况表': 911957,
 '2020年4月保险业经营情况表': 906375,
 '2020年3月保险业经营情况表': 900170,
 '2020年2月保险业经营情况表': 896051,
 '2020年1月保险业经营情况表': 894619}

In [22]:
driver = webdriver.Firefox(firefox_profile=fp)
for key,val in dic2.items():    
    try:
        print("The page is about {key}\n".format(key=key))
        get_one_file(driver,key,val)
        sleep(3)
    except:
        print(key,'\tFailed')
        get_one_file(driver,key,val)
        #continue

The page is about 2020年12月保险业经营情况表

Success
The page is about 2020年11月保险业经营情况表

Success
The page is about 2020年10月保险业经营情况表

2020年10月保险业经营情况表 	Failed
Success
The page is about 2020年9月保险业经营情况表

2020年9月保险业经营情况表 	Failed
Success
The page is about 2020年8月保险业经营情况表

2020年8月保险业经营情况表 	Failed
Success
The page is about 2020年7月保险业经营情况表

2020年7月保险业经营情况表 	Failed
Success
The page is about 2020年6月保险业经营情况表

2020年6月保险业经营情况表 	Failed
Success
The page is about 2020年5月保险业经营情况表

2020年5月保险业经营情况表 	Failed
Success
The page is about 2020年4月保险业经营情况表

Success
The page is about 2020年3月保险业经营情况表

2020年3月保险业经营情况表 	Failed
Success
The page is about 2020年2月保险业经营情况表

2020年2月保险业经营情况表 	Failed
Success
The page is about 2020年1月保险业经营情况表

2020年1月保险业经营情况表 	Failed
Success


### Data Processing

In [14]:
import pandas as pd
import numpy as np


In [9]:
file = sorted(os.listdir(raw_data_path)[:-1],key=lambda x: os.path.getmtime(os.path.join(raw_data_path, x)))

In [11]:
processed_path = '/mnt/d/data_science/Insurance_report/data/processed'

In [22]:
def clean_column(columns_list):
    """
    这个代码负责将表格的列名称清洗干净（就是将一些二级标题、三级标题都转换成完整的名称：{一级标题}-{二级标题}-{三级标题})
    如果标题是一级标题 -> 一级标题
    如果标题是二级标题 -> {一级标题}-{二级标题}
    如果标题是三级标题 -> {一级标题}-{二级标题}-{三级标题}

    这么一做，每一个表的列名称就不会混淆

    """

    column_data = pd.DataFrame({'column_name':[i.replace(" ", '') for i in columns_list]})
    # 做一级填充
    column_data['class_1'] = column_data['column_name'].apply(lambda x: len(re.findall(pattern='[0-9]+、', string=x)) !=1)#[len(re.findall(pattern='[0-9]+、', string=temp_column)) !=1 for temp_column in columns_list]
    column_data['class_1_name'] = column_data['column_name']
    column_data.loc[column_data['class_1']== False, 'class_1_name'] = np.nan
    column_data['class_1_name'] = column_data['class_1_name'].ffill()
    # column_data.loc[column_data['column_name'] != column_data['class_1_name'],'class_1_name'] = \
    #     column_data.loc[column_data['column_name'] != column_data['class_1_name'],:].apply(lambda x: f"{x['class_1_name']}__{x['column_name']}", axis=1)
    column_data.loc[column_data['class_1'] == False,'class_1_name'] = \
        column_data.loc[column_data['class_1'] == False,:].apply(lambda x: f"{x['class_1_name']}__{x['column_name']}", axis=1)
    # 做二级填充

    column_data['class_2'] = column_data['column_name'].apply(lambda x: len(re.findall(pattern='（[0-9]+）', string=x))!=1)

    column_data['class_2_name'] = column_data['class_1_name']
    column_data.loc[column_data['class_2'] == False, 'class_2_name'] = np.nan
    column_data['class_2_name'] = column_data['class_2_name'].ffill()

    column_data.loc[column_data['class_2'] == False, 'class_2_name'] = column_data.loc[column_data['class_2'] == False, :].apply(lambda x: f"{x['class_2_name']}__{x['column_name']}", axis=1)

    return column_data['class_2_name'].tolist()

In [62]:
def get_one_data(i):
    try:
        data = pd.read_excel(raw_data_path + i,engine = 'openpyxl',skiprows=2).drop('Unnamed: 0',axis = 1).dropna()
    except:
        data = pd.read_excel(raw_data_path + i,engine = 'xlrd',skiprows=2).drop('Unnamed: 0',axis = 1).dropna()
    data = data.set_index('项目').T.reset_index(drop=True)
    columns_list = data.columns.tolist()
    data.columns = clean_column(columns_list)
    data['info'] = '{i}'.format(i = i[:-6])
    return data
#data

In [60]:
dd = pd.DataFrame()
for i in file:
    try:
        tmp = get_one_data(i)
        dd = dd.append(tmp)
    except:
        print(i)

2021年6月保险业经营情况表.xlsx


In [ ]:
data = pd.read_excel(raw_data_path + '2021年6月保险业经营情况表.xlsx',engine = 'openpyxl',skiprows=2).drop(['Unnamed: 0','Unnamed: 3'],axis = 1).dropna()
data = data.set_index('项目').T.reset_index(drop=True)
#data
columns_list = data.columns.tolist()
data.columns = clean_column(columns_list)
data['info'] = '2021年6月保险业经营情况'
data
dd = dd.append(data)

In [41]:
import xlrd
A =xlrd.open_workbook(raw_data_path + '2021年5月保险业经营情况表.xlsx')

In [73]:
dd.to_csv('../data/industry_new.csv',index=False)